Hypothèses communes

* Domaine 1D (x\in[0,L]), (t\in[0,T])
* Schrödinger linéaire non relativiste
  [
  i\partial_t\psi=-\tfrac12\partial_{xx}\psi+V(x)\psi
  ]
* (\psi) complexe, (V) réel
* Conservation attendue : (|\psi(t)|*{L^2}=|\psi(0)|*{L^2})

---

## CAS 1 — Conditions périodiques, potentiel nul

### Énoncé

[
\begin{cases}
i\partial_t\psi=-\tfrac12\partial_{xx}\psi, & x\in(0,L),\ t>0\
\psi(0,t)=\psi(L,t),\quad \psi_x(0,t)=\psi_x(L,t)\
\psi(x,0)=\psi_0(x)
\end{cases}
]

---

## Résolution par différences finies

### Discrétisation spatiale

Grille uniforme
[
x_j=jh,\quad h=L/N,\quad j=0,\dots,N-1
]

Laplacien périodique
[
(\partial_{xx}\psi)*j \approx \frac{\psi*{j+1}-2\psi_j+\psi_{j-1}}{h^2}
]

Système semi-discret
[
i\frac{d\Psi}{dt}=-\tfrac12 A\Psi
]

où (A) est une matrice circulante symétrique réelle.

---

### Schéma temporel — Crank–Nicolson (unitaire)

[
\left(I+\tfrac{i\Delta t}{4}A\right)\Psi^{n+1}
==============================================

\left(I-\tfrac{i\Delta t}{4}A\right)\Psi^{n}
]

* Ordre 2 en temps
* Conservation exacte de la norme discrète
* Coût (O(N^2)) sans diagonalisation

```python
import numpy as np

L, T = 2*np.pi, 1.0
N, dt = 256, 1e-3
h = L/N
x = np.linspace(0, L, N, endpoint=False)

psi = np.exp(-10*(x-np.pi)**2)*np.exp(1j*x)

e = np.ones(N)
A = (-2*np.diag(e) + np.diag(e[:-1],1) + np.diag(e[:-1],-1)) / h**2
A[0,-1] = A[-1,0] = 1/h**2

I = np.eye(N)
M1 = I + 0.25j*dt*A
M2 = I - 0.25j*dt*A

Nt = int(T/dt)
for _ in range(Nt):
    psi = np.linalg.solve(M1, M2 @ psi)
```

Complexité : (O(N^3)) naïf, (O(N^2)) avec solveur bande
Stabilité : inconditionnelle
Conservation : (\ell^2) exacte

---

## Résolution spectrale (méthode de Fourier)

### Décomposition

Base de Fourier complexe
[
\psi(x,t)=\sum_{k\in\mathbb{Z}}\hat\psi_k(t)e^{i2\pi kx/L}
]

Valeurs propres
[
-\partial_{xx}\to \lambda_k=(2\pi k/L)^2
]

---

### Système modal

[
i\frac{d\hat\psi_k}{dt}=\tfrac12\lambda_k\hat\psi_k
]

Solution exacte
[
\hat\psi_k(t)=e^{-i\frac{\lambda_k}{2}t}\hat\psi_k(0)
]

---

### Implémentation FFT

```python
from scipy.fft import fft, ifft, fftfreq

k = fftfreq(N, d=h)
lam = (2*np.pi*k)**2

psi_hat = fft(psi)
psi_hat *= np.exp(-0.5j*lam*T)
psi = ifft(psi_hat)
```

* Coût : (O(N\log N))
* Erreur : spectrale (exponentielle si (\psi_0) régulière)
* Conservation : norme (L^2) machine

---

## CAS 2 — Conditions périodiques avec potentiel (V(x))

### Énoncé

[
i\partial_t\psi=-\tfrac12\partial_{xx}\psi+V(x)\psi
]

---

## Différences finies

Semi-discret
[
i\frac{d\Psi}{dt}=\left(-\tfrac12 A+V_d\right)\Psi
]

où (V_d=\mathrm{diag}(V(x_j)))

Schéma Crank–Nicolson
[
\left(I+\tfrac{i\Delta t}{2}H\right)\Psi^{n+1}
==============================================

\left(I-\tfrac{i\Delta t}{2}H\right)\Psi^n
]

```python
V = 0.5*np.cos(x)
H = -0.5*A + np.diag(V)

M1 = I + 0.5j*dt*H
M2 = I - 0.5j*dt*H

for _ in range(Nt):
    psi = np.linalg.solve(M1, M2 @ psi)
```

* Hamiltonien hermitien
* Schéma unitaire
* Coût élevé pour grands (N)

---

## Spectral — Split-Step Fourier (Strang)

Décomposition
[
H = T + V,\quad
T=-\tfrac12\partial_{xx}
]

Approximation
[
e^{-iH\Delta t}
\approx
e^{-iV\Delta t/2}
e^{-iT\Delta t}
e^{-iV\Delta t/2}
]

Ordre 2 en temps, symplectique.

```python
Vx = np.exp(-0.5j*V*dt)
Tk = np.exp(-0.5j*lam*dt)

for _ in range(Nt):
    psi *= Vx
    psi_hat = fft(psi)
    psi_hat *= Tk
    psi = ifft(psi_hat)
    psi *= Vx
```

* Conservation norme (L^2)
* Coût : (O(N\log N))
* Méthode standard en physique quantique numérique

---

## CAS 3 — Conditions de Dirichlet homogènes

### Énoncé

[
\begin{cases}
i\partial_t\psi=-\tfrac12\partial_{xx}\psi+V(x)\psi\
\psi(0,t)=\psi(L,t)=0\
\psi(x,0)=\psi_0(x)
\end{cases}
]

---

## Différences finies

Grille intérieure (x_i=ih,\ i=1,\dots,N)

Laplacien Dirichlet classique
Schéma Crank–Nicolson identique au cas périodique, sans circularité.

---

## Spectral — Base sinus

Décomposition
[
\psi(x,t)=\sum_{k=1}^N \hat\psi_k(t)\sin\left(\frac{k\pi x}{L}\right)
]

Valeurs propres
[
\lambda_k=\left(\frac{k\pi}{L}\right)^2
]

Sans potentiel
[
\hat\psi_k(t)=e^{-i\lambda_k t/2}\hat\psi_k(0)
]

```python
from scipy.fftpack import dst, idst

x = np.linspace(0, L, N+2)[1:-1]
psi0 = np.sin(np.pi*x/L)

psi_hat = dst(psi0, type=1)
k = np.arange(1, N+1)
lam = (k*np.pi/L)**2

psi_hat *= np.exp(-0.5j*lam*T)
psi = idst(psi_hat, type=1)/(2*N)
```

Avec potentiel : split-step + DST.

---

## Points de vigilance CC3

* Toujours vérifier la conservation de la norme
* Crank–Nicolson = unitaire seulement si (H) hermitien
* Split-step valide si (T) et (V) traités exactement
* FFT uniquement en périodique
* DST uniquement pour Dirichlet homogène

Si tu veux la suite logique :

* **problème Schrödinger avec solution manufacturée + calcul d’erreur**,
* **comparaison DF vs spectral sur la dispersion numérique**,
* **cas onde plane vs paquet gaussien (interprétation physique)**.
